In [5]:
import src.config as config

In [2]:
import hopsworks

In [14]:
import pandas as pd

In [19]:
#connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

#connect to the feature store
feature_store = project.get_feature_store()

#connect to the feature group
feature_group = feature_store.get_feature_group(
    name= config.FEATURE_GROUP_NAME,
    version= config.FEATURE_GROUP_VERSION
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/845972
Connected. Call `.close()` to terminate connection gracefully.


In [20]:
#get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version= config.FEATURE_GROUP_VERSION
)

In [21]:
ft_data, _ = feature_view.training_data(
    description= 'month_features_target_bogota'
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.64s) 
2024-07-08 15:04:54,664 WARNING: VersionWarning: Incremented version to `3`.



In [22]:
ft_data.head()

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,target
0,111.534431,110.934189,110.354187,110.399513,110.300919,110.512520,110.274162,109.628159,110.123192,111.742798,112.188606,113.490685,113.407112
1,113.440956,113.152206,113.309822,113.079880,114.028374,114.500031,114.951485,116.761017,117.933739,119.055634,119.664001,120.437683,120.936508
2,178.755020,179.348602,179.641891,179.930359,181.030563,181.194321,180.772842,181.979538,183.006836,183.099579,183.963821,184.313538,185.154480
3,93.595520,94.650833,94.937325,96.869972,99.408813,100.000000,100.978447,101.456337,101.592758,102.447937,104.124565,106.097946,105.861473
4,223.262375,221.240677,219.212555,216.934647,215.434418,215.712753,216.541412,216.197662,214.089737,215.156860,216.450302,216.389542,214.269150


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [24]:
import joblib as jb

In [27]:
import lightgbm as lgb

In [25]:
# Preparacion de los datos
x = ft_data.drop(['target'], axis=1) #Features
y = ft_data['target'] #target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 28)

In [28]:
model = lgb.LGBMRegressor()

model.fit(
    x_train, y_train,
    eval_set=[(x_test, y_test)],
    eval_metric='rmse',    
)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000324 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 756
[LightGBM] [Info] Number of data points in the train set: 185, number of used features: 12
[LightGBM] [Info] Start training from score 164.654569
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

LGBMRegressor()

In [29]:
y_pred = model.predict(x_test)
predictions = model.predict(x_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
test_mae = mean_absolute_error(y_test, predictions)

print(f'{test_mae=:.4f}')
print(f'RMSE: {rmse=:.4f}')

2024-07-08 15:10:22,627 WARNING: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

test_mae=2.0253
RMSE: rmse=3.1370


In [30]:
from src.paths import ML_MODELS

In [31]:
jb.dump(model, rf'{ML_MODELS}/ml_bogota.pkl')

['D:\\0_Respaldo\\0_Proyectos_2024\\ML_proyects\\ipvn_predictor\\data\\ml_models/ml_bogota.pkl']

In [32]:
#Hopsworks model schema
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

In [33]:
input_schema= Schema(x_train)
output_schema= Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema= output_schema)

In [35]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name='monthly_ipvn_bogota',
    metrics={'test_mae': test_mae},
    description='LightGBM regressor with NO hiper parameters TUNNING',
    input_example=x_train.sample(),
    model_schema=model_schema
)

Connected. Call `.close()` to terminate connection gracefully.


In [37]:
### Save the model in the hopsworks
model_path = str(ML_MODELS/'ml_bogota.pkl')
model.save(model_path)

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/80961 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/232 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1086 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/845972/models/monthly_ipvn_bogota/1


Model(name: 'monthly_ipvn_bogota', version: 1)